In [3]:
import pandas as pd
import numpy as np
import csv
# import xgboost as xgb
from tqdm import tqdm
from sklearn.svm import SVC
from sklearn import preprocessing#, decomposition, model_selection, metrics, pipeline
from nltk import word_tokenize
from sklearn.model_selection import cross_val_score
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [2]:
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')
# sample = pd.read_csv('../input/sample_submission.csv')
print(train.shape)
test.shape

(106445, 3)


(26610, 2)

In [17]:
# delete document with empty content
rm_idx = []
for i in range(len(test)):
    if len(test.Text[i]) <= 20:
        print(test.Text[i,])
        rm_idx.append(i)

# train = train.iloc[train.idx,]
# print(train.shape)

KeyError: (3018,)

In [16]:
print(rm_idx)

[4235, 4238, 4265, 8749, 8756, 8818, 18393, 18427, 19020, 25067, 27378, 27768, 31485, 31660, 31671, 31738, 40951, 41030, 41129, 41136, 41152, 41171, 41275, 41586, 42016, 45758, 45759, 45760, 45761, 50687, 51670, 51710, 52231, 52347, 52943, 66800, 70787, 78766, 92399, 92637, 94982, 97814, 98045, 101651, 101850, 101862, 102065, 102117, 102203]


In [68]:
# train = train.drop(train.index[rm_idx])
# train.reset_index(drop=True, inplace=True)
# train.shape

(106396, 3)

In [4]:
# load the GloVe vectors in a dictionary:
embeddings_index = {}
f = open('D:/Master of Data Science/18S2-FIT5149 - Applied Data Analysis/Assignment/glove.42B.300d.txt', encoding="utf8")
for line in tqdm(f):
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

1917494it [02:21, 13571.78it/s]


Found 1917494 word vectors.


In [ ]:
import WordNet Lemmatizer from nltk 
from nltk.stem import WordNetLemmatizer 
wordnet_lemmatizer = WordNetLemmatizer() 
lines_with_lemmas=[] #stop words contain the set of stop words 
for line in lines: 
temp_line=[] 
for word in lines: 
    temp_line.append (wordnet_lemmatizer.lemmatize(word)) 
    string=’ ‘ 
    lines_with_lemmas.append(string.join(temp_line)) 
    lines=lines_with_lemmas

In [1]:
# this function creates a scaled vector for an entire document
def doc2vec(s):
    words = str(s).lower()
    words = word_tokenize(words) # tokenize the doc
    words = [w for w in words if not w in stop_words] # remove stop words
    words = [w for w in words if w.isalpha()] # remove numbers

    # create an array excluding stopwords and numbers
    M = []
    for w in words:
        try:
            M.append(embeddings_index[w])
        except:
            continue
    M = np.array(M)

    # for error document, convert into a 300d vector of 0
    v = M.sum(axis=0)
    if type(v) != np.ndarray:
        return np.zeros(300)
        
    return v / np.sqrt((v ** 2).sum())

In [5]:
xtrain = train.Text.values
ytrain = train.label
xtest = test.Text.values

# create sentence vectors using the above function for training and validation set
xtrain_glove = [sent2vec(x) for x in tqdm(xtrain)]
xtest_glove = [sent2vec(x) for x in tqdm(xtest)]

100%|███████████████████████████████████████████████████████████████████████████| 26610/26610 [00:50<00:00, 524.76it/s]


In [6]:
train_colname = []
test_colname = []
for i in range(300):
    name = "v" + str(i)
    train_colname.append(name)
    test_colname.append(name)
# print(train_colname)

In [7]:
train_output = pd.DataFrame.from_records(xtrain_glove)
test_output = pd.DataFrame.from_records(xtest_glove)

train_output.columns = train_colname
test_output.columns = test_colname

# train_output['id'] = train.ID
train_output['label'] = ytrain

# test_output['id'] = test.ID

In [8]:
train_output.columns

Index(['v0', 'v1', 'v2', 'v3', 'v4', 'v5', 'v6', 'v7', 'v8', 'v9',
       ...
       'v291', 'v292', 'v293', 'v294', 'v295', 'v296', 'v297', 'v298', 'v299',
       'label'],
      dtype='object', length=301)

In [11]:
# train_output = train_output[new_traincol]
# test_output = test_output[new_testcol]
print(train_output.shape)
print(test_output.shape)

(106445, 301)
(26610, 300)


In [15]:
len(train_output.label.unique())

23

In [10]:
train_output.iloc[4235,]

v0        0
v1        0
v2        0
v3        0
v4        0
v5        0
v6        0
v7        0
v8        0
v9        0
v10       0
v11       0
v12       0
v13       0
v14       0
v15       0
v16       0
v17       0
v18       0
v19       0
v20       0
v21       0
v22       0
v23       0
v24       0
v25       0
v26       0
v27       0
v28       0
v29       0
         ..
v271      0
v272      0
v273      0
v274      0
v275      0
v276      0
v277      0
v278      0
v279      0
v280      0
v281      0
v282      0
v283      0
v284      0
v285      0
v286      0
v287      0
v288      0
v289      0
v290      0
v291      0
v292      0
v293      0
v294      0
v295      0
v296      0
v297      0
v298      0
v299      0
label    C1
Name: 4235, Length: 301, dtype: object

In [9]:
train_output.to_csv('./train_glove_final.csv',index=False)
test_output.to_csv('./test_glove_final.csv',index=False)

In [79]:
train_output.label

0          C1
1          C1
2          C1
3          C1
4          C1
5          C1
6          C1
7          C1
8          C1
9          C1
10         C1
11         C1
12         C1
13         C1
14         C1
15         C1
16         C1
17         C1
18         C1
19         C1
20         C1
21         C1
22         C1
23         C1
24         C1
25         C1
26         C1
27         C1
28         C1
29         C1
         ... 
106366    C23
106367    C23
106368    C23
106369    C23
106370    C23
106371    C23
106372    C23
106373    C23
106374    C23
106375    C23
106376    C23
106377    C23
106378    C23
106379    C23
106380    C23
106381    C23
106382    C23
106383    C23
106384    C23
106385    C23
106386    C23
106387    C23
106388    C23
106389    C23
106390    C23
106391    C23
106392    C23
106393    C23
106394    C23
106395    C23
Name: label, Length: 106396, dtype: object